In [63]:
%%configure -f
{"conf":{"spark.sql.legacy.parquet.datetimeRebaseModeInRead":"CORRECTED",
         "spark.sql.legacy.parquet.int96RebaseModeInWrite":"CORRECTED",
        "spark.sql.legacy.parquet.datetimeRebaseModeInWrite":"CORRECTED"}
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1631660312588_0005,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1631660312588_0005,spark,idle,Link,Link,✔


In [64]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@79fe77f9


In [65]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._


In [7]:
val schema_cnae = "cod_cnae INT, descricao_cnae STRING"
val df_cnae = (spark.read
               .option("header","false")
               .option("sep",";")
               .option("encoding","ISO-8859-1")
               .schema(schema_cnae)
               .csv("s3://datalake-claudemir-649165755582/F.K03200$Z.D10710.CNAE.csv"))

df_cnae.printSchema()
df_cnae.show(15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schema_cnae: String = cod_cnae INT, descricao_cnae STRING
df_cnae: org.apache.spark.sql.DataFrame = [cod_cnae: int, descricao_cnae: string]
root
 |-- cod_cnae: integer (nullable = true)
 |-- descricao_cnae: string (nullable = true)

+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
|  111301|    Cultivo de arroz|
|  111302|    Cultivo de milho|
|  111303|    Cultivo de trigo|
|  111399|Cultivo de outros...|
|  112101|Cultivo de algodã...|
|  112102|     Cultivo de juta|
|  112199|Cultivo de outras...|
|  113000|Cultivo de cana-d...|
|  114800|     Cultivo de fumo|
|  115600|     Cultivo de soja|
|  116401| Cultivo de amendoim|
|  116402| Cultivo de girassol|
|  116403|   Cultivo de mamona|
|  116499|Cultivo de outras...|
|  119901|  Cultivo de abacaxi|
+--------+--------------------+
only showing top 15 rows



In [8]:
val schema_municipios = "cod_municipio INT, descricao_municipio STRING"
val df_municipios = (spark.read
               .option("header","false")
               .option("sep",";")
               .option("encoding","ISO-8859-1")
               .schema(schema_municipios)
               .csv("s3://datalake-claudemir-649165755582/F.K03200$Z.D10710.MUNIC.csv"))

df_municipios.printSchema()
df_municipios.show(15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schema_municipios: String = cod_municipio INT, descricao_municipio STRING
df_municipios: org.apache.spark.sql.DataFrame = [cod_municipio: int, descricao_municipio: string]
root
 |-- cod_municipio: integer (nullable = true)
 |-- descricao_municipio: string (nullable = true)

+-------------+--------------------+
|cod_municipio| descricao_municipio|
+-------------+--------------------+
|            1|       GUAJARA-MIRIM|
|            2|ALTO ALEGRE DOS P...|
|            3|         PORTO VELHO|
|            4|             BURITIS|
|            5|           JI-PARANA|
|            6|         CHUPINGUAIA|
|            7|           ARIQUEMES|
|            8|             CUJUBIM|
|            9|              CACOAL|
|           10|          NOVA UNIAO|
|           11|       PIMENTA BUENO|
|           12|             PARECIS|
|           13|             VILHENA|
|           14|PIMENTEIRAS DO OESTE|
|           15|                JARU|
+-------------+--------------------+
only showing top 15 ro

In [9]:
(df_cnae
 .write
 .mode("overwrite")
 .parquet(s"s3://datalake-claudemir-649165755582/modulo3-refined/cnae")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
(df_municipios
 .write
 .mode("overwrite")
 .parquet(s"s3://datalake-claudemir-649165755582/modulo3-refined/municipios")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
val schema_estabelecimentos = """
cnpj_basico INT, cnpj_ordem INT, cnpj_dv INT, id_matriz INT, nome_fantasia STRING, situacao_cadastral INT, data_situacao_cadastral STRING,
motivo_situacao_cadastral INT, nome_cidade_exterior STRING, id_pais INT, data_inicio_atividade STRING, cnae_fiscal_principal INT, cnae_fiscal_secundaria INT,
tipo_logradouro STRING, logradouro STRING, num_estabelecimento INT, complemento STRING, bairro STRING, cep STRING, uf STRING, cod_municipio INT,
ddd1 STRING, telefone1 STRING, ddd2 STRING, telefone2 STRING, ddd_fax STRING, fax STRING, email STRING, sit_especial STRING, data_sit_especial STRING"""
val df_estabelecimentos = (spark.read
               .option("header","false")
               .option("sep",";")
               .option("encoding","ISO-8859-1")
               .option("escape","\"")
               .schema(schema_estabelecimentos)
               .csv("s3://datalake-claudemir-649165755582/modulo3/"))

val df_estabelecimentos_with_date = (df_estabelecimentos
                                     .withColumn("ano_situacao_cadastral",substring(col("data_situacao_cadastral"),0,4))
                                     .withColumn("mes_situacao_cadastral",substring(col("data_situacao_cadastral"),5,2))
                                     .withColumn("dia_situacao_cadastral",substring(col("data_situacao_cadastral"),7,2))
                                     .withColumn("new_data_situacao_cadastral",concat_ws("-",
                                                                                         col("ano_situacao_cadastral"),
                                                                                         col("mes_situacao_cadastral"),
                                                                                         col("dia_situacao_cadastral")
                                                                                        ).cast(DateType))
                                     .drop("ano_situacao_cadastral","mes_situacao_cadastral","dia_situacao_cadastral","data_situacao_cadastral")
                                     .withColumnRenamed("new_data_situacao_cadastral","data_situacao_cadastral")
                                     .withColumn("ano_sit_especial",substring(col("data_sit_especial"),0,4))
                                     .withColumn("mes_sit_especial",substring(col("data_sit_especial"),5,2))
                                     .withColumn("dia_sit_especial",substring(col("data_sit_especial"),7,2))
                                     .withColumn("new_data_sit_especial",concat_ws("-",
                                                                                   col("ano_sit_especial"),
                                                                                   col("mes_sit_especial"),
                                                                                   col("dia_sit_especial")
                                                                                  ).cast(DateType))
                                     .drop("ano_sit_especial","mes_sit_especial","dia_sit_especial","data_sit_especial")
                                     .withColumnRenamed("new_data_sit_especial","data_sit_especial")
                                     .withColumn("ano_inicio_atividade",substring(col("data_inicio_atividade"),0,4))
                                     .withColumn("mes_inicio_atividade",substring(col("data_inicio_atividade"),5,2))
                                     .withColumn("dia_inicio_atividade",substring(col("data_inicio_atividade"),7,2))
                                     .withColumn("new_data_inicio_atividade",concat_ws("-",
                                                                                   col("ano_inicio_atividade"),
                                                                                   col("mes_inicio_atividade"),
                                                                                   col("dia_inicio_atividade")
                                                                                  ).cast(DateType))
                                     .drop("ano_inicio_atividade","mes_inicio_atividade","dia_inicio_atividade","data_inicio_atividade")
                                     .withColumnRenamed("new_data_inicio_atividade","data_inicio_atividade")  
                                    )



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schema_estabelecimentos: String =
"
cnpj_basico INT, cnpj_ordem INT, cnpj_dv INT, id_matriz INT, nome_fantasia STRING, situacao_cadastral INT, data_situacao_cadastral STRING,
motivo_situacao_cadastral INT, nome_cidade_exterior STRING, id_pais INT, data_inicio_atividade STRING, cnae_fiscal_principal INT, cnae_fiscal_secundaria INT,
tipo_logradouro STRING, logradouro STRING, num_estabelecimento INT, complemento STRING, bairro STRING, cep STRING, uf STRING, cod_municipio INT,
ddd1 STRING, telefone1 STRING, ddd2 STRING, telefone2 STRING, ddd_fax STRING, fax STRING, email STRING, sit_especial STRING, data_sit_especial STRING"
df_estabelecimentos: org.apache.spark.sql.DataFrame = [cnpj_basico: int, cnpj_ordem: int ... 28 more fields]
df_estabelecimentos_with_date: org.apache.spark.sql.DataFrame = [cnpj_basico: int, cnpj_ordem: int ... 28 more fields]


df_estabelecimentos.printSchema()
df_estabelecimentos.show(15)

In [18]:
df_estabelecimentos.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res21: Long = 49198427


In [52]:
df_estabelecimentos_with_date.select("data_sit_especial","data_situacao_cadastral","data_inicio_atividade").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------------+---------------------+
|data_sit_especial|data_situacao_cadastral|data_inicio_atividade|
+-----------------+-----------------------+---------------------+
|             null|             2016-11-30|           2016-11-30|
|             null|             2021-02-12|           2016-11-30|
|             null|             2016-12-16|           2016-11-30|
|             null|             2021-03-03|           2016-11-30|
|             null|             2016-11-30|           2016-11-30|
|             null|             2016-11-30|           2016-11-30|
|             null|             2018-07-31|           2016-11-30|
|             null|             2016-11-30|           2016-11-30|
|             null|             2017-04-30|           2016-11-30|
|             null|             2021-04-13|           2016-11-30|
+-----------------+-----------------------+---------------------+
only showing top 10 rows



In [53]:
df_estabelecimentos_with_date.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- id_matriz: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- id_pais: integer (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: integer (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- num_estabelecimento: integer (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- cod_municipio: integer (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- telefone1: string (nullable = true)
 |-- ddd2: string (nullable = true)
 |-- telefone2: strin

In [67]:
(df_estabelecimentos_with_date
 .write
 .mode("overwrite")
 .parquet(s"s3://datalake-claudemir-649165755582/modulo3-refined/estabelecimentos")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# ----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [71]:
val df_municipios_parquet = spark.read.parquet("s3://datalake-claudemir-649165755582/modulo3-refined/municipios").withColumnRenamed("cod_municipio","municipio_cod_municipio")
val df_cnae_parquet = spark.read.parquet("s3://datalake-claudemir-649165755582/modulo3-refined/cnae")
val df_estabelecimentos_parquet = spark.read.parquet("s3://datalake-claudemir-649165755582/modulo3-refined/estabelecimentos")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_municipios_parquet: org.apache.spark.sql.DataFrame = [municipio_cod_municipio: int, descricao_municipio: string]
df_cnae_parquet: org.apache.spark.sql.DataFrame = [cod_cnae: int, descricao_cnae: string]
df_estabelecimentos_parquet: org.apache.spark.sql.DataFrame = [cnpj_basico: int, cnpj_ordem: int ... 28 more fields]


In [12]:
// cod_cnae = cnae_fiscal_principal ; cod_municipio = cod_municipio

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
val joined_df = (
    df_estabelecimentos_parquet
    .join(
        broadcast(df_municipios_parquet),
        df_estabelecimentos_parquet.col("cod_municipio") === df_municipios_parquet.col("municipio_cod_municipio"),
        "left"
    )
    .join(
        broadcast(df_cnae_parquet),
        df_estabelecimentos_parquet.col("cnae_fiscal_principal") === df_cnae_parquet.col("cod_cnae"),
        "left"
    )
//     .drop("municipio_cod_municipio")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

joined_df: org.apache.spark.sql.DataFrame = [cnpj_basico: int, cnpj_ordem: int ... 32 more fields]


In [73]:
(joined_df
 .write
 .mode("overwrite")
 .parquet(s"s3://datalake-claudemir-649165755582/modulo3-refined/joined_tables"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# ----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [76]:
val df_joined_tables = spark.read.parquet("s3://datalake-claudemir-649165755582/modulo3-refined/joined_tables").drop("municipio_cod_municipio")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_joined_tables: org.apache.spark.sql.DataFrame = [cnpj_basico: int, cnpj_ordem: int ... 31 more fields]


In [77]:
df_joined_tables.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- id_matriz: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- id_pais: integer (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: integer (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- num_estabelecimento: integer (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- cod_municipio: integer (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- telefone1: string (nullable = true)
 |-- ddd2: string (nullable = true)
 |-- telefone2: strin

In [82]:
(df_joined_tables
 .filter("situacao_cadastral = 2")
 .groupBy("cod_cnae").agg(count(col("cnpj_basico")).alias("num_empresas"))
 .orderBy(desc_nulls_first("num_empresas"))
 .select(col("cod_cnae"),col("num_empresas"))
 .show(10)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+
|cod_cnae|num_empresas|
+--------+------------+
| 4781400|      991316|
| 9602501|      754449|
| 4399103|      458648|
| 7319002|      453914|
| 4712100|      444980|
| 5611203|      419118|
| 5611201|      363814|
| 5620104|      320047|
| 8112500|      287024|
| 8219999|      266308|
+--------+------------+
only showing top 10 rows



In [87]:
(df_joined_tables
 .filter("situacao_cadastral != 2")
 .filter("uf = 'SP' ")
 .count()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res17: Long = 7966472


In [122]:
df_joined_tables.select("cod_municipio").filter("descricao_municipio = 'BELO HORIZONTE'").distinct().show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|cod_municipio|
+-------------+
|         4123|
+-------------+



In [127]:
// 13 - errada
(df_joined_tables
 .filter("situacao_cadastral = 2")
  .filter("cod_cnae = 6204000")
//  .filter("descricao_cnae like '%Consultoria em tecnologia da informação%'")
 .filter("cod_municipio = 4123")
//  .select("situacao_cadastral","descricao_cnae","descricao_municipio")
//  .show(15)
  .count()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res65: Long = 1114


In [118]:
(df_joined_tables
 .filter("situacao_cadastral = 2")
 .filter("cod_cnae = 6204000")
 .filter("descricao_municipio = 'BELO HORIZONTE'")
//  .select("cod_cnae","situacao_cadastral","descricao_cnae","descricao_municipio")
//  .show(15)
  .count()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res50: Long = 1114


In [116]:
(df_joined_tables
 .select("cod_cnae","descricao_cnae")
 .filter("descricao_cnae like '%informação%'")
 .groupBy("cod_cnae","descricao_cnae").count()
 .show(20)
 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+------+
|cod_cnae|      descricao_cnae| count|
+--------+--------------------+------+
| 6209100|Suporte técnico, ...|112784|
| 6319400|Portais, provedor...| 31539|
| 6399200|Outras atividades...| 19416|
| 6204000|Consultoria em te...| 63751|
+--------+--------------------+------+



In [102]:
(df_joined_tables
 .filter("nome_fantasia like 'IGTI'")
 .filter("descricao_municipio = 'BELO HORIZONTE'")
 .select("cod_cnae","nome_fantasia","descricao_municipio","descricao_cnae")
 .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------+-------------------+--------------------+
|cod_cnae|nome_fantasia|descricao_municipio|      descricao_cnae|
+--------+-------------+-------------------+--------------------+
| 8532500|         IGTI|     BELO HORIZONTE|Educação superior...|
+--------+-------------+-------------------+--------------------+



In [106]:
(df_joined_tables
 .filter("data_inicio_atividade >= to_date('2020-01-01','yyyy-MM-dd')")
 .select("cod_cnae","nome_fantasia","data_inicio_atividade")
 .count()
 //  .show(10)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res37: Long = 6314456
